# **[Context Managers and Python's with Statement](https://realpython.com/python-with-statement/)**

### **1. Managing Resources in Python**

In [1]:
file = open("hello.txt", "w")
file.write("Hello, World!")
file.close()

In [4]:
# Safely open the file
file = open("hello.txt", "w")

try:
    file.write("Hello, World!")
finally:
    # Make sure to close the file after using it
    file.close()

In [5]:
# Safely open the file
file = open("hello.txt", "w")

try:
    file.write("Hello, World!")
except Exception as e:
    print(f"An error occurred while writing to the file: {e}")
finally:
    # Make sure to close the file after using it
    file.close()

#### **[io — Core tools for working with streams](https://docs.python.org/3/library/io.html#raw-i-o)**
- Text I/O
- Binary I/O
- Raw I/O

In [6]:
with open("hello.txt", mode="w") as file:
    file.write("Hello, World!")

In [10]:
with open("input.txt", mode="w") as in_file, open("output.txt", "w") as out_file:
    # Read content from input.txt
    in_file.write("Hello, There!")
    # Transform the content
    # Write the transformed content to output.txt
    out_file.write("Hello, Sanggoo!")
    pass

### **2. Using the Python with Statement**
> #### **2.1. Working With Files**

In [11]:
with open("hello.txt", mode="w") as file:
    file.write("Hello, World!")

In [13]:
file = open("hello.txt", mode="w")

with file:
    file.write("Hello, World!")

# When you try to run a second with, however, you get a ValueError because your file is already closed.
with file:
    file.write("Welcome to Real Python!")

ValueError: I/O operation on closed file.

In [16]:
file = open("hello.txt", mode="w")

with file:
    file.write("Hello, World!")

file = open("hello.txt", mode="w")

with file:
    file.write("Welcome to Real Python!")

In the above approach, **the file is explicitly opened outside the with block**

So

- Here, each with statement handles opening and closing the file independently.
- This means that for each block, the file is opened, written to, and then closed.

In [15]:
with open("hello.txt", mode="w") as file:
    file.write("Hello, World!")

with open("hello.txt", mode="w") as file:
    file.write("Welcome to Real Python!")

**Path** is a class that represents **concrete paths to physical files** in your computer. Calling .open() on a Path object that points to a physical file opens it just like open() would do. So, Path.open() works similarly to open(), but the file path is automatically provided by the Path object you call the method on.

In [17]:
import pathlib

file_path = pathlib.Path("hello.txt")

with file_path.open("w") as file:
    file.write("Hello, World!")

In [20]:
import pathlib
import logging

file_path = pathlib.Path("hello.txt")

try:
    with file_path.open(mode="w") as file:
        file.write("Hello, What a Wonderful World!")

except OSError as error:
    logging.error("Writing to file %s failed due to: %s", file_path, error)

> #### **2.2. Traversing Directories**

In [21]:
import os

with os.scandir(".") as entries:
    for entry in entries:
        print(entry.name, "->", entry.stat().st_size, "bytes")

.config -> 4096 bytes
input.txt -> 13 bytes
output.txt -> 15 bytes
hello.txt -> 30 bytes
sample_data -> 4096 bytes


> #### **2.3. Performing High-Precision Calculations**

In [30]:
from decimal import Decimal, localcontext

with localcontext() as ctx:
    ctx.prec = 50                 # 정밀도(precision)를 42로 설정
    print(f'정밀도(precision)를 42로 설정한 역수 : {Decimal("1") / Decimal("42")}')


print(f'정밀도(precision)를 default로 설정한 역수 : {Decimal("1") / Decimal("42")}')

정밀도(precision)를 42로 설정한 역수 : 0.023809523809523809523809523809523809523809523809524
정밀도(precision)를 default로 설정한 역수 : 0.02380952380952380952380952381


> #### **2.4. Handling Locks in Multithreaded Programs**

In [37]:
import threading

balance_lock = threading.Lock()

# Use the try ... finally pattern
balance_lock.acquire()
try:
    # Update the account balance here
    pass
finally:
    balance_lock.release()

# Use the with pattern
with balance_lock:
    # Update the account balance here
    pass

계좌 잔액 업데이트를 수행하는 간단한 코드 예제로 with 블록을 사용하여 잔액을 업데이트하는 한 개의 함수만 사용하고, 단 두 개의 스레드로 작업을 수행.
**특정 계좌의 온라인 송금시 송금이 완료될 떄까지 locking하여야 함**

In [39]:
import threading

# 공유 변수인 계좌 잔액
balance = 0
balance_lock = threading.Lock()

def update_balance(amount):
    global balance
    # with 패턴을 사용하여 임계 영역에 안전하게 진입
    with balance_lock:
        balance += amount
        print(f"Thread {threading.current_thread().name}: Balance updated to {balance}")

# 스레드를 생성하고 실행
def main():
    threads = [
        threading.Thread(target=update_balance, args=(100,), name="Thread-A"),
        threading.Thread(target=update_balance, args=(-50,), name="Thread-B")
    ]

    # 스레드 시작
    for t in threads:
        t.start()

    # 모든 스레드가 종료될 때까지 대기
    for t in threads:
        t.join()

    # 최종 잔액 출력
    print(f"Final balance: {balance}")

if __name__ == "__main__":
    main()

Thread Thread-A: Balance updated to 100
Thread Thread-B: Balance updated to 50
Final balance: 50


> #### **2.5. Testing for Exceptions With pytest**

In [45]:
import pytest
1 / 0

ZeroDivisionError: division by zero

In [42]:
with pytest.raises(ZeroDivisionError):
    1 / 0

In [46]:
favorites = {"fruit": "apple", "pet": "dog"}
favorites["car"]

KeyError: 'car'

In [47]:
with pytest.raises(KeyError):
    favorites["car"]

In [48]:
import pytest

with pytest.raises(ZeroDivisionError):
    4 / 2

Failed: DID NOT RAISE <class 'ZeroDivisionError'>

In [49]:
with pytest.raises(ZeroDivisionError) as exc:
    1 / 0

assert str(exc.value) == "division by zero"

- VS code에서 "math_functions.py"와 "test_math_functions.py"를 만들고 DOS prompt에서

>> C:\python>**pytest test_math_functions.py**
======================================================== test session starts ========================================================
platform win32 -- Python 3.12.3, pytest-8.1.1, pluggy-1.4.0
rootdir: C:\python
collected 4 items

>> test_math_functions.py ....                                                                                                    [100%]
========================================================= 4 passed in 0.03s =========================================================

In [50]:
%%writefile math_functions.py
# math_functions.py

def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero.")
    return a / b


Writing math_functions.py


In [51]:
# test_math_functions.py
import pytest
from math_functions import add, subtract, multiply, divide

def test_add():
    assert add(1, 2) == 3
    assert add(-1, 1) == 0
    assert add(-1, -1) == -2

def test_subtract():
    assert subtract(10, 5) == 5
    assert subtract(-1, 1) == -2
    assert subtract(2, 3) == -1

def test_multiply():
    assert multiply(3, 4) == 12
    assert multiply(-1, 1) == -1
    assert multiply(0, 10) == 0

def test_divide():
    assert divide(10, 2) == 5
    assert divide(5, 2) == 2.5
    with pytest.raises(ValueError):
        divide(10, 0)

> ### **3. Summarizing the with Statement’s Advantages**
>> Using the with statement consistently can improve the general quality of your code and make it safer by preventing resource leak problems.

> ### **4. Using the async with Statement**

In [52]:
# site_checker_v0.py
# VS code py 파일로 실행 --> python site_checker_v0.py

import aiohttp
import asyncio

async def check(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print(f"{url}: status -> {response.status}")
            html = await response.text()
            print(f"{url}: type -> {html[:17].strip()}")

async def main():
    await asyncio.gather(
        check("https://realpython.com"),
        check("https://pycoders.com"),
    )

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [53]:
import aiohttp
import asyncio

async def check(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print(f"{url}: status -> {response.status}")
            html = await response.text()
            print(f"{url}: type -> {html[:17].strip()}")

async def main():
    await asyncio.gather(
        check("https://realpython.com"),
        check("https://pycoders.com"),
    )

# Jupyter 노트북이나 다른 이벤트 루프가 이미 실행 중인 환경에서 사용
loop = asyncio.get_event_loop()
if loop.is_running():
    await main()
else:
    loop.run_until_complete(main())

https://realpython.com: status -> 200
https://realpython.com: type -> <!doctype html>
https://pycoders.com: status -> 200
https://pycoders.com: type -> <!doctype html>
